In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from langchain_groq.chat_models import ChatGroq
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve API key from the environment variables
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Available Models
groq_models = {
    "llama3-70b": "llama3-70b-8192",
    "mixtral": "mixtral-8x7b-32768",
    "gemma-7b": "gemma-7b-it",
    "llama3.1-70b": "llama-3.1-70b-versatile",
    "llama3-8b": "llama3-8b-8192",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gemma-9b": "gemma2-9b-it"
}
model = groq_models["llama3-70b"]

# Constants
green = "\x1b[38;2;152;251;152m"
red = "\x1b[38;2;255;0;0m"
reset = "\x1b[0m"
time = 10
offset = 100
folders = ["LAYING", "SITTING", "STANDING", "WALKING", "WALKING_DOWNSTAIRS", "WALKING_UPSTAIRS"]
classes = {"WALKING": 1, "WALKING_UPSTAIRS": 2, "WALKING_DOWNSTAIRS": 3, "SITTING": 4, "STANDING": 5, "LAYING": 6}

combined_dir = os.path.join("C:\\Users\\arpit\\OneDrive\\Desktop\\es335-24-fall-assignment-1\\UCI HAR Dataset\\Combined")

X_train = []
y_train = []

dataset_dir = os.path.join(combined_dir, "Train")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir, folder))

    for file in files:
        df = pd.read_csv(os.path.join(dataset_dir, folder, file), sep=",", header=0)
        df = df[offset:offset + time * 50]
        X_train.append(df.values)
        y_train.append(classes[folder])

X_test = []
y_test = []

dataset_dir = os.path.join(combined_dir, "Test")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir, folder))
    for file in files:
        df = pd.read_csv(os.path.join(dataset_dir, folder, file), sep=",", header=0)
        df = df[offset:offset + time * 50]
        X_test.append(df.values)
        y_test.append(classes[folder])

# Concatenate the training and testing data
X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

# Split the data into training and testing sets. Change the seed value to obtain different random splits.
seed = 4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y)

print("Training data shape: ", X_train.shape)
print("Testing data shape: ", X_test.shape)

categories = list(classes.keys())

# Train prompt template
train_prompt = lambda data, category: f"""
You do not need to output anything for this query.
Example data for "{category}":
{data}
"""

# Test prompt template
test_prompt = lambda data: f"""
You will be provided 3-axis accelerometer data from one of the following activities:
    1) WALKING
    2) LAYING
    3) WALKING_UPSTAIRS
    4) WALKING_DOWNSTAIRS
    5) SITTING
    6) STANDING
You have to output only the name of the predicted activity and nothing else.
DATA:
{data}
"""


Training data shape:  (126, 500, 3)
Testing data shape:  (54, 500, 3)


In [2]:
llm = ChatGroq(model=model, api_key=GROQ_API_KEY, temperature=0)

correct_count = 0

for i in range(len(X_test)):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if(ans==categories[y_test[i]-1]):
        correct_count+=1
        color = green
    else:
        color = red
    print(f"{i}) Actual: {categories[y_test[i]-1]}\n{color}Predicted: {ans}{reset}")

print(f"""
Total: {len(X_test)}
Correct: {correct_count}
Accuracy: {correct_count/len(X_test)}
""")

0) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
1) Actual: WALKING
Predicted: WALKING
2) Actual: WALKING_UPSTAIRS
Predicted: WALKING
3) Actual: STANDING
Predicted: WALKING
4) Actual: STANDING
Predicted: WALKING
5) Actual: WALKING
Predicted: WALKING_UPSTAIRS
6) Actual: WALKING
Predicted: WALKING
7) Actual: STANDING
Predicted: WALKING
8) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
9) Actual: WALKING_UPSTAIRS
Predicted: WALKING
10) Actual: LAYING
Predicted: WALKING
11) Actual: STANDING
Predicted: WALKING
12) Actual: LAYING
Predicted: WALKING
13) Actual: STANDING
Predicted: WALKING
14) Actual: LAYING
Predicted: WALKING_UPSTAIRS
15) Actual: WALKING
Predicted: WALKING
16) Actual: LAYING
Predicted: WALKING
17) Actual: STANDING
Predicted: WALKING
18) Actual: WALKING_UPSTAIRS
Predicted: WALKING_UPSTAIRS
19) Actual: STANDING
Predicted: WALKING
20) Actual: SITTING
Predicted: WALKING
21) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
22) Actual: WALKING_UPSTAIRS
Predicted: WALKING_UPSTAIRS
23) A

In [3]:
llm = ChatGroq(model=model, api_key=GROQ_API_KEY, temperature=0)

print(llm.invoke("""
You will be provided examples for 3-axis accelerometer data from one of the following activities:
    1) WALKING
    2) LAYING
    3) WALKING_UPSTAIRS
    4) WALKING_DOWNSTAIRS
    5) SITTING
    6) STANDING
You have to learn from these examples and then you will be tested against test dataset.
You do not need to output anything for this query.
""").content)

for i in range(len(X_train)):
    query = train_prompt(X_train[i],categories[y_train[i]-1])
    response = llm.invoke(query)
    #print(response.content)

I'm ready to learn from the provided examples. Please go ahead and provide the 3-axis accelerometer data for each of the activities (WALKING, LAYING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING). I'll learn from these examples and prepare to be tested against the test dataset.


In [4]:
correct_count = 0

for i in range(len(X_test)):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if(ans==categories[y_test[i]-1]):
        correct_count+=1
        color = green
    else:
        color = red
    print(f"{i}) Actual: {categories[y_test[i]-1]}\n{color}Predicted: {ans}{reset}")

print(f"""Total: {len(X_test)}
Correct: {correct_count}
Accuracy: {correct_count/len(X_test)}
""")

0) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
1) Actual: WALKING
Predicted: WALKING
2) Actual: WALKING_UPSTAIRS
Predicted: WALKING
3) Actual: STANDING
Predicted: WALKING
4) Actual: STANDING
Predicted: WALKING
5) Actual: WALKING
Predicted: WALKING_UPSTAIRS
6) Actual: WALKING
Predicted: WALKING
7) Actual: STANDING
Predicted: WALKING
8) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
9) Actual: WALKING_UPSTAIRS
Predicted: WALKING
10) Actual: LAYING
Predicted: WALKING
11) Actual: STANDING
Predicted: WALKING
12) Actual: LAYING
Predicted: WALKING
13) Actual: STANDING
Predicted: WALKING
14) Actual: LAYING
Predicted: WALKING_UPSTAIRS
15) Actual: WALKING
Predicted: WALKING
16) Actual: LAYING
Predicted: WALKING
17) Actual: STANDING
Predicted: WALKING
18) Actual: WALKING_UPSTAIRS
Predicted: WALKING_UPSTAIRS
19) Actual: STANDING
Predicted: WALKING
20) Actual: SITTING
Predicted: WALKING
21) Actual: WALKING_DOWNSTAIRS
Predicted: WALKING
22) Actual: WALKING_UPSTAIRS
Predicted: WALKING_UPSTAIRS
23) A

In [5]:
x_min = np.min(X_test)
x_max = np.max(X_test)
y_min = np.min(y_test)
y_max = np.max(y_test)

X_test = (x_max-x_min)*np.random.random(len(X_test)) + x_min
y_test = np.random.randint(y_min,y_max+1,len(X_test))

correct_count = 0

for i in range(len(X_test)):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if(ans==categories[y_test[i]-1]):
        correct_count+=1
        color = green
    else:
        color = red
    print(f"{i}) Actual: {categories[y_test[i]-1]}\n{color}Predicted: {ans}{reset}")

print(f"""Total: {len(X_test)}
Correct: {correct_count}
Accuracy: {correct_count/len(X_test)}
""")

0) Actual: WALKING_DOWNSTAIRS
Predicted: STANDING
1) Actual: WALKING_DOWNSTAIRS
Predicted: STANDING
2) Actual: SITTING
Predicted: STANDING
3) Actual: WALKING
Predicted: STANDING
4) Actual: WALKING
Predicted: STANDING
5) Actual: WALKING_UPSTAIRS
Predicted: STANDING
6) Actual: WALKING
Predicted: STANDING
7) Actual: WALKING
Predicted: STANDING
8) Actual: LAYING
Predicted: STANDING
9) Actual: WALKING_DOWNSTAIRS
Predicted: STANDING
10) Actual: STANDING
Predicted: STANDING
11) Actual: SITTING
Predicted: STANDING
12) Actual: LAYING
Predicted: STANDING
13) Actual: WALKING
Predicted: STANDING
14) Actual: WALKING
Predicted: STANDING
15) Actual: WALKING_DOWNSTAIRS
Predicted: STANDING
16) Actual: STANDING
Predicted: STANDING
17) Actual: SITTING
Predicted: STANDING
18) Actual: WALKING
Predicted: STANDING
19) Actual: SITTING
Predicted: STANDING
20) Actual: WALKING_UPSTAIRS
Predicted: STANDING
21) Actual: LAYING
Predicted: STANDING
22) Actual: WALKING_DOWNSTAIRS
Predicted: STANDING
23) Actual: SITTIN

In [ ]:
import pandas as pd
import numpy as np
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
API_KEY = os.getenv('GROQ_API_KEY')

# Initialize the LLM
llm = ChatGroq(model='llama3-70b', api_key=API_KEY)

# Define the activity descriptions
activity_descriptions = {
    "WALKING": "Moving forward with alternating steps.",
    "SITTING": "Seated with minimal movement.",
    "STANDING": "Standing still with slight body shifts.",
    "LAYING": "Lying down with minimal body movement.",
    "WALKING_UPSTAIRS": "Moving upward on stairs.",
    "WALKING_DOWNSTAIRS": "Descending stairs with alternating steps."
}

def create_prompt(features):
    prompt = (
        "You are an expert in human activity recognition. Based on the following sensor data, "
        "determine the most likely human activity being performed:\n"
        f"Mean acceleration in x-axis: {features['mean_x']:.2f} m/s^2\n"
        f"Mean acceleration in y-axis: {features['mean_y']:.2f} m/s^2\n"
        f"Mean acceleration in z-axis: {features['mean_z']:.2f} m/s^2\n"
        f"Standard deviation in x-axis: {features['std_x']:.2f} m/s^2\n"
        f"Standard deviation in y-axis: {features['std_y']:.2f} m/s^2\n"
        f"Standard deviation in z-axis: {features['std_z']:.2f} m/s^2\n"
        "Possible activities include walking, sitting, standing, laying, walking upstairs, "
        "and walking downstairs. Please state the most likely activity."
    )
    return prompt

# Classify the activity based on the features using Zero-Shot Learning
def classify_activity(features):
    prompt = create_prompt(features)
    try:
        # Use the LLM to generate a response
        response = llm.generate(prompt)
        print(f"Generated Prompt:\n{prompt}")  # For debugging
        print(f"Full Model Response: {response}")  # For debugging
        # Extract and return the response text
        prediction = response.strip().split("\n")[0] if response else "Unknown"
        return prediction if prediction in activity_descriptions else "Unknown"
    except Exception as e:
        print(f"Error during model generation: {e}")
        return "Unknown"

# Feature extraction
def extract_features(data):
    df = pd.DataFrame(data.reshape(-1, 3), columns=['accx', 'accy', 'accz'])
    
    features = {
        'mean_x': df['accx'].mean(),
        'mean_y': df['accy'].mean(),
        'mean_z': df['accz'].mean(),
        'std_x': df['accx'].std(),
        'std_y': df['accy'].std(),
        'std_z': df['accz'].std()
    }
    return features

def main():
    # Randomly select 10 samples from X_train and corresponding y_train
    num_samples = 10
    random_indices = np.random.choice(X_train.shape[0], size=num_samples, replace=False)
    
    # Correctly indexing rows using .iloc[]
    X_samples = X_train.iloc[random_indices]
    y_true = y_train.iloc[random_indices]

    predictions = []
    for data in X_samples.values:  # .values returns the underlying numpy array
        # Extract features from each sample
        features = extract_features(data)
        
        # Classify activity
        predicted_activity = classify_activity(features)
        predictions.append(predicted_activity)
    
    # Convert predictions to a numpy array
    predictions = np.array(predictions)
    
    # Calculate and print accuracy
    accuracy = np.mean(predictions == y_true.values) * 100
    print("Predictions:", predictions)
    print("True labels:", y_true.values) 
    print(f"Accuracy: {accuracy:.2f}%")

if __name__ == "__main__":
    main()


In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from langchain_groq.chat_models import ChatGroq
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve API key from the environment variables
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Available Models
groq_models = {
    "llama3-70b": "llama3-70b-8192",
    "mixtral": "mixtral-8x7b-32768",
    "gemma-7b": "gemma-7b-it",
    "llama3.1-70b": "llama-3.1-70b-versatile",
    "llama3-8b": "llama3-8b-8192",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gemma-9b": "gemma2-9b-it"
}
model = groq_models["llama3-70b"]

# Constants
time = 10
offset = 100
folders = ["LAYING", "SITTING", "STANDING", "WALKING", "WALKING_DOWNSTAIRS", "WALKING_UPSTAIRS"]
classes = {"WALKING": 1, "WALKING_UPSTAIRS": 2, "WALKING_DOWNSTAIRS": 3, "SITTING": 4, "STANDING": 5, "LAYING": 6}

combined_dir = os.path.join("C:\\Users\\arpit\\OneDrive\\Desktop\\es335-24-fall-assignment-1\\UCI HAR Dataset\\Combined")

X_train = []
y_train = []

dataset_dir = os.path.join(combined_dir, "Train")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir, folder))
    for file in files:
        df = pd.read_csv(os.path.join(dataset_dir, folder, file), sep=",", header=0)
        df = df[offset:offset + time * 50]
        X_train.append(df.values)
        y_train.append(classes[folder])

X_test = []
y_test = []

dataset_dir = os.path.join(combined_dir, "Test")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir, folder))
    for file in files:
        df = pd.read_csv(os.path.join(dataset_dir, folder, file), sep=",", header=0)
        df = df[offset:offset + time * 50]
        X_test.append(df.values)
        y_test.append(classes[folder])

# Concatenate the training and testing data
X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

# Split the data into training and testing sets
seed = 4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y)

categories = list(classes.keys())

# Define the ChatGroq LLM
llm = ChatGroq(model=model, api_key=GROQ_API_KEY, temperature=0)

# Few-Shot Learning Example
print("Few-Shot Learning:")
few_shot_examples = 3  # Using only 3 examples per activity for training

# Train prompt template
train_prompt = lambda data, category: f"""
You do not need to output anything for this query.
Example data for "{category}":
{data}
"""

# Test prompt template
test_prompt = lambda data: f"""
You will be provided 3-axis accelerometer data from one of the following activities:
    1) WALKING
    2) LAYING
    3) WALKING_UPSTAIRS
    4) WALKING_DOWNSTAIRS
    5) SITTING
    6) STANDING
You have to output only the name of the predicted activity and nothing else.
DATA:
{data}
"""

# Training with Few-Shot Learning
for i in range(few_shot_examples):
    for j in range(len(folders)):
        query = train_prompt(X_train[i + j * few_shot_examples], categories[y_train[i + j * few_shot_examples] - 1])
        llm.invoke(query)

# Testing the model
correct_count = 0
for i in range(len(X_test)):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if ans == categories[y_test[i] - 1]:
        correct_count += 1
    print(f"{i}) Actual: {categories[y_test[i] - 1]}, Predicted: {ans}")

print(f"""
Total: {len(X_test)}
Correct: {correct_count}
Accuracy: {correct_count/len(X_test)}
""")

# Zero-Shot Learning Example
print("Zero-Shot Learning:")

correct_count = 0
for i in range(len(X_test)):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if ans == categories[y_test[i] - 1]:
        correct_count += 1
    print(f"{i}) Actual: {categories[y_test[i] - 1]}, Predicted: {ans}")

print(f"""
Total: {len(X_test)}
Correct: {correct_count}
Accuracy: {correct_count/len(X_test)}
""")


Few-Shot Learning:
0) Actual: WALKING_DOWNSTAIRS, Predicted: WALKING
1) Actual: WALKING, Predicted: WALKING
2) Actual: WALKING_UPSTAIRS, Predicted: WALKING
3) Actual: STANDING, Predicted: WALKING
4) Actual: STANDING, Predicted: WALKING
5) Actual: WALKING, Predicted: WALKING_UPSTAIRS
6) Actual: WALKING, Predicted: WALKING
7) Actual: STANDING, Predicted: WALKING
8) Actual: WALKING_DOWNSTAIRS, Predicted: WALKING
9) Actual: WALKING_UPSTAIRS, Predicted: WALKING
10) Actual: LAYING, Predicted: WALKING
11) Actual: STANDING, Predicted: WALKING
12) Actual: LAYING, Predicted: WALKING
13) Actual: STANDING, Predicted: WALKING
14) Actual: LAYING, Predicted: WALKING_UPSTAIRS
15) Actual: WALKING, Predicted: WALKING
16) Actual: LAYING, Predicted: WALKING
17) Actual: STANDING, Predicted: WALKING
18) Actual: WALKING_UPSTAIRS, Predicted: WALKING_UPSTAIRS
19) Actual: STANDING, Predicted: WALKING
20) Actual: SITTING, Predicted: WALKING
21) Actual: WALKING_DOWNSTAIRS, Predicted: WALKING
22) Actual: WALKING_U

In [15]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from langchain_groq.chat_models import ChatGroq
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve API key from the environment variables
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Available Models
groq_models = {
    "llama3-70b": "llama3-70b-8192",
    "mixtral": "mixtral-8x7b-32768",
    "gemma-7b": "gemma-7b-it",
    "llama3.1-70b": "llama-3.1-70b-versatile",
    "llama3-8b": "llama3-8b-8192",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gemma-9b": "gemma2-9b-it"
}
model = groq_models["llama3-70b"]

# Constants
time = 10
offset = 100
folders = ["LAYING", "SITTING", "STANDING", "WALKING", "WALKING_DOWNSTAIRS", "WALKING_UPSTAIRS"]
classes = {"WALKING": 1, "WALKING_UPSTAIRS": 2, "WALKING_DOWNSTAIRS": 3, "SITTING": 4, "STANDING": 5, "LAYING": 6}

combined_dir = os.path.join("C:\\Users\\arpit\\OneDrive\\Desktop\\es335-24-fall-assignment-1\\UCI HAR Dataset\\Combined")

X_train = []
y_train = []

dataset_dir = os.path.join(combined_dir, "Train")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir, folder))
    for file in files:
        df = pd.read_csv(os.path.join(dataset_dir, folder, file), sep=",", header=0)
        df = df[offset:offset + time * 50]
        X_train.append(df.values)
        y_train.append(classes[folder])

X_test = []
y_test = []

dataset_dir = os.path.join(combined_dir, "Test")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir, folder))
    for file in files:
        df = pd.read_csv(os.path.join(dataset_dir, folder, file), sep=",", header=0)
        df = df[offset:offset + time * 50]
        X_test.append(df.values)
        y_test.append(classes[folder])

# Concatenate the training and testing data
X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

# Split the data into training and testing sets
seed = 4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y)

categories = list(classes.keys())

# Define the ChatGroq LLM
llm = ChatGroq(model=model, api_key=GROQ_API_KEY, temperature=0)

# Few-Shot Learning Example (5 examples per class)
print("Few-Shot Learning:")
few_shot_examples_per_class = 5  # Number of examples per class

# Train prompt template
train_prompt = lambda data, category: f"""
You do not need to output anything for this query.
Example data for "{category}":
{data}
"""

# Test prompt template
test_prompt = lambda data: f"""
You will be provided 3-axis accelerometer data from one of the following activities:
    1) WALKING
    2) LAYING
    3) WALKING_UPSTAIRS
    4) WALKING_DOWNSTAIRS
    5) SITTING
    6) STANDING
You have to output only the name of the predicted activity and nothing else.
DATA:
{data}
"""

# Training with Few-Shot Learning (5 examples per class)
used_indices = set()  # To track used examples and avoid duplicates
for folder in folders:
    class_indices = [i for i, label in enumerate(y_train) if label == classes[folder]]
    selected_indices = class_indices[:few_shot_examples_per_class]  # Select the first 5 examples for this class

    for index in selected_indices:
        if index not in used_indices:
            query = train_prompt(X_train[index], categories[y_train[index] - 1])
            llm.invoke(query)
            used_indices.add(index)  # Mark this example as used

# Testing the model (Few-Shot Learning)
correct_count_fs = 0
for i in range(len(X_test)):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if ans == categories[y_test[i] - 1]:
        correct_count_fs += 1
    print(f"{i}) Actual: {categories[y_test[i] - 1]}, Predicted: {ans}")



# Zero-Shot Learning Example (10 examples)
print("Zero-Shot Learning:")
zero_shot_examples = 10  # Using only 10 examples for testing

correct_count_zs = 0
for i in range(zero_shot_examples):
    query = test_prompt(X_test[i])
    ans = llm.invoke(query).content
    if ans == categories[y_test[i] - 1]:
        correct_count_zs += 1
    print(f"{i}) Actual: {categories[y_test[i] - 1]}, Predicted: {ans}")

accuracy_zs = correct_count_zs / zero_shot_examples
print(f"Zero-Shot Learning Accuracy: {accuracy_zs}")
accuracy_fs = correct_count_fs / len(X_test)
print(f"Few-Shot Learning Accuracy: {accuracy_fs}")


Few-Shot Learning:
0) Actual: WALKING_DOWNSTAIRS, Predicted: WALKING
1) Actual: WALKING, Predicted: WALKING
2) Actual: WALKING_UPSTAIRS, Predicted: WALKING
3) Actual: STANDING, Predicted: WALKING
4) Actual: STANDING, Predicted: WALKING
5) Actual: WALKING, Predicted: WALKING_UPSTAIRS
6) Actual: WALKING, Predicted: WALKING
7) Actual: STANDING, Predicted: WALKING
8) Actual: WALKING_DOWNSTAIRS, Predicted: WALKING
9) Actual: WALKING_UPSTAIRS, Predicted: WALKING
10) Actual: LAYING, Predicted: WALKING
11) Actual: STANDING, Predicted: WALKING
12) Actual: LAYING, Predicted: WALKING
13) Actual: STANDING, Predicted: WALKING
14) Actual: LAYING, Predicted: WALKING_UPSTAIRS
15) Actual: WALKING, Predicted: WALKING
16) Actual: LAYING, Predicted: WALKING
17) Actual: STANDING, Predicted: WALKING
18) Actual: WALKING_UPSTAIRS, Predicted: WALKING_UPSTAIRS
19) Actual: STANDING, Predicted: WALKING
20) Actual: SITTING, Predicted: WALKING
21) Actual: WALKING_DOWNSTAIRS, Predicted: WALKING
22) Actual: WALKING_U

: 